## Rerun of acquiring DOIs using the Crossref API with the python library 'habanero' 
(conda installed v0.6)
check with first ten 



code for basic install and a simple search of the works using a query and a filter



In [153]:
import pandas as pd
from habanero import Crossref, exceptions
import time

### First bring in the existing list of publictaions (using the columns needed)

In [154]:
df1 = pd.read_csv('./A3out_CrossRef_BRC_FuzzyMatched_07Aug19.csv', index_col=0)
df1.shape

(2378, 18)

In [155]:
df1.head(2)

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match,CrossRefTitle_low,DOI_fuzzyRatio,TitleMatch01,Title_fuzzyRatio,Title_partialRatio,Complete_fuzzyRatio,Complete_partialRatio,good_list
0,10.1186/s12881-014-0095-4,1125,"&amp; , fenwick al, goos jac, rankin j, lord h...",{'title': ['Apparently synonymous substitution...,apparently synonymous substitutions in fgfr2 a...,10.1186/s12881-014-0095-4,['Apparently synonymous substitutions in FGFR2...,Apparently synonymous substitutions in FGFR2af...,False,True,apparently synonymous substitutions in fgfr2af...,100.0,0,99.0,99.0,51.0,99.0,True
1,10.1183/13993003.00321-2016,1996,", pattinson kt, turner mr. a wider pathologica...",{'title': ['A wider pathological network under...,a wider pathological network underlying breath...,10.1183/13993003.00321-2016,['A wider pathological network underlying brea...,A wider pathological network underlying breath...,True,True,a wider pathological network underlying breath...,100.0,1,100.0,100.0,73.0,100.0,True


In [156]:
#make dataframe from only those missing DOIs
#df1[~df1.doi.isna()]  gives those with DOIs (1440 as reported)
df1[df1.good_list==False].shape


(379, 18)

### Therefore identifying that 353 of the 2378 publications without good DOI matches
### _however, we don't know any of them are correct_ so it will best to acquire all top matches from CrossRef


In [157]:
df1mini =df1[df1.good_list==False].iloc[5:8]
df1mini

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match,CrossRefTitle_low,DOI_fuzzyRatio,TitleMatch01,Title_fuzzyRatio,Title_partialRatio,Complete_fuzzyRatio,Complete_partialRatio,good_list
26,10.3389/fmicb.2015.01016,1642,"adland e, klenerman p, goulder p, matthews pc....",{'title': ['Review of cytomegalovirus coinfect...,ongoing burden of disease and mortality from h...,10.1002/rmv.1907,['Review of cytomegalovirus coinfection in HIV...,Review of cytomegalovirus coinfection in HIV-i...,False,False,review of cytomegalovirus coinfection in hiv-i...,40.0,0,47.0,52.0,32.0,52.0,False
33,NaN,1639,"ahmed s, brennan l, eppig j, price cc, lamar m...",{'title': ['Comparative study between depressi...,visuoconstructional impairment in subtypes of ...,10.1111/j.1442-2018.2011.00666.x,['Comparative study between depression in Kore...,Comparative study between depression in Korean...,False,False,comparative study between depression in korean...,NaN,0,47.0,56.0,38.0,49.0,False
55,10.1097/BOR.0b013e3283620163,563,"al-mossawi mh, ridley a, kiedel s, bowness p. ...",{'title': ['Gamma delta T cells in non-immune ...,"the role of natural killer cells, gamma delta ...",10.1002/iid3.18,['Gamma delta T cells in non-immune patients d...,Gamma delta T cells in non-immune patients dur...,False,False,gamma delta t cells in non-immune patients dur...,42.0,0,47.0,54.0,33.0,55.0,False


In [158]:
cr = Crossref()

In [159]:
#Crossref(api_key = "168421")
# set a mailto address
Crossref(mailto = "laurence.brown@ndcn.ox.ac.uk")

< Crossref 
URL: https://api.crossref.org
KEY: None
MAILTO: laurence.brown@ndcn.ox.ac.uk
>

In [160]:
#cr.filter_details()

In [161]:
'''['DOI', 'URL', 'alternative-id', 'author','indexed', 'is-referenced-by-count',
       'language', 'license', 'link',  'published-online', 'published-print', 'publisher',
       'reference', 'reference-count', 'references-count', 'relation', 'score',
       'short-container-title', 'source', 'title', 'type', 'update-to']
'''
#select=['title', 'link','URL', 'author','is-referenced-by-count','alternative-id', 'author','indexed',],

"['DOI', 'URL', 'alternative-id', 'author','indexed', 'is-referenced-by-count',\n       'language', 'license', 'link',  'published-online', 'published-print', 'publisher',\n       'reference', 'reference-count', 'references-count', 'relation', 'score',\n       'short-container-title', 'source', 'title', 'type', 'update-to']\n"

In [162]:
crossref_test =cr.works(query=df1mini.complete[26]
             ,select=['title','DOI'],
                      filter={'from_pub_date':'2011-01-01','until_pub_date':'2018-12-31','type':'journal-article'},
                      limit=1)

In [163]:
crossref_test['message']['items'][0]['DOI'],crossref_test['message']['items'][0]['title'][0]

('10.3389/fmicb.2015.01016',
 'Ongoing burden of disease and mortality from HIV/CMV coinfection in Africa in the antiretroviral therapy era')

In [164]:
def CrossRefDoiRetry(df):
    try:
        search = cr.works(query=df.complete
             ,select=['title','DOI'],filter={'from_pub_date':'2011-01-01','until_pub_date':'2018-12-31','type':'journal-article'},
                      limit=1)
        time.sleep(1)
        return search['message']['items'][0]['DOI'], search['message']['items'][0]['title'][0]
    except:
        return None
    finally:
        time.sleep(1)
        pass

In [165]:
#df1x.columns


In [166]:
df1mini[['retry_API','retry_title']]= df1mini.apply(CrossRefDoiRetry, axis=1, result_type='expand')

In [167]:
df1mini

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match,CrossRefTitle_low,DOI_fuzzyRatio,TitleMatch01,Title_fuzzyRatio,Title_partialRatio,Complete_fuzzyRatio,Complete_partialRatio,good_list,retry_API,retry_title
26,10.3389/fmicb.2015.01016,1642,"adland e, klenerman p, goulder p, matthews pc....",{'title': ['Review of cytomegalovirus coinfect...,ongoing burden of disease and mortality from h...,10.1002/rmv.1907,['Review of cytomegalovirus coinfection in HIV...,Review of cytomegalovirus coinfection in HIV-i...,False,False,review of cytomegalovirus coinfection in hiv-i...,40.0,0,47.0,52.0,32.0,52.0,False,10.3389/fmicb.2015.01016,Ongoing burden of disease and mortality from H...
33,NaN,1639,"ahmed s, brennan l, eppig j, price cc, lamar m...",{'title': ['Comparative study between depressi...,visuoconstructional impairment in subtypes of ...,10.1111/j.1442-2018.2011.00666.x,['Comparative study between depression in Kore...,Comparative study between depression in Korean...,False,False,comparative study between depression in korean...,NaN,0,47.0,56.0,38.0,49.0,False,10.1016/j.jalz.2014.05.929,VISUOCONSTRUCTIONAL IMPAIRMENT IN SUBTYPES OF ...
55,10.1097/BOR.0b013e3283620163,563,"al-mossawi mh, ridley a, kiedel s, bowness p. ...",{'title': ['Gamma delta T cells in non-immune ...,"the role of natural killer cells, gamma delta ...",10.1002/iid3.18,['Gamma delta T cells in non-immune patients d...,Gamma delta T cells in non-immune patients dur...,False,False,gamma delta t cells in non-immune patients dur...,42.0,0,47.0,54.0,33.0,55.0,False,10.1097/bor.0b013e3283620163,"The role of natural killer cells, gamma delta ..."


In [168]:
df_retry =df1[df1.good_list==False]

In [169]:
df_retry[['DOI_retry_API','title_retry_API']]= df_retry.apply(CrossRefDoiRetry, axis=1, result_type='expand')

C:\Users\labrown\Miniconda3\envs\biblio_tools\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [178]:
df_retry.to_csv('./A4out_badlist_API_retry(07Aug19).csv')

In [179]:
df_retry

,DOI,ID,complete,crossref_API_out,csv_post_title,DOI_API,title,title_API,title_match,DOI_match,CrossRefTitle_low,DOI_fuzzyRatio,TitleMatch01,Title_fuzzyRatio,Title_partialRatio,Complete_fuzzyRatio,Complete_partialRatio,good_list,DOI_retry_API,title_retry_API
5,NaN,1312,"g w, rane , harston g w, rane n, shaya g, t...",{'title': ['Imaging-based selection in acute i...,imaging biomarkers in acute ischemic stroke tr...,10.1111/j.1749-6632.2012.06732.x,['Imaging-based selection in acute ischemic st...,Imaging-based selection in acute ischemic stro...,False,False,imaging-based selection in acute ischemic stro...,NaN,0,68.0,72.0,43.0,64.0,False,10.3174/ajnr.a4208,Imaging Biomarkers in Acute Ischemic Stroke Tr...
7,10.1016/j.atherosclerosis.2012.03.036,62,"kylintireas i, craig s, nethononda r, kohle...",{'title': ['Obstructive sleep apnea in childre...,atherosclerosis and arterial stiffness in obst...,10.1002/lary.23844,['Obstructive sleep apnea in children'],Obstructive sleep apnea in children,False,False,obstructive sleep apnea in children,40.0,0,42.0,77.0,23.0,77.0,False,10.1016/j.atherosclerosis.2012.03.036,Atherosclerosis and arterial stiffness in obst...
10,NaN,168,"pagnamenta at, murray je, yoon g, akha es, ...",{'title': ['High-frequency sensorineural heari...,a novel nonsense cdk5rap2 mutation in a somali...,10.1002/lary.25544,['High-frequency sensorineural hearing loss in...,High-frequency sensorineural hearing loss in c...,False,False,high-frequency sensorineural hearing loss in c...,NaN,0,41.0,60.0,24.0,62.0,False,10.1002/ajmg.a.35558,A novel nonsenseCDK5RAP2mutation in a Somali c...
20,10.1016/j.fertnstert.2014.12.123,1920,"åukaszuk k, pukszta s, wells d, cybulska c, l...",{'title': ['Validation of a semiconductor next...,routine use of next-generation sequencing for ...,10.1002/pd.4665,['Validation of a semiconductor next-generatio...,Validation of a semiconductor next-generation ...,False,False,validation of a semiconductor next-generation ...,34.0,0,62.0,64.0,42.0,64.0,False,10.1016/j.fertnstert.2014.12.123,Routine use of next-generation sequencing for ...
21,NaN,1023,"abdel-hady m, bansal s, davison sm, brown m, t...",{'title': ['Chronic hepatitis C in children - ...,treatment of chronic viral hepatitis c in chil...,10.1111/j.1365-2893.2011.01456.x,['Chronic hepatitis C in children - review of ...,Chronic hepatitis C in children - review of na...,False,False,chronic hepatitis c in children - review of na...,NaN,0,55.0,59.0,35.0,56.0,False,10.1136/archdischild-2013-304601,Treatment of chronic viral hepatitis C in chil...
26,10.3389/fmicb.2015.01016,1642,"adland e, klenerman p, goulder p, matthews pc....",{'title': ['Review of cytomegalovirus coinfect...,ongoing burden of disease and mortality from h...,10.1002/rmv.1907,['Review of cytomegalovirus coinfection in HIV...,Review of cytomegalovirus coinfection in HIV-i...,False,False,review of cytomegalovirus coinfection in hiv-i...,40.0,0,47.0,52.0,32.0,52.0,False,10.3389/fmicb.2015.01016,Ongoing burden of disease and mortality from H...
33,NaN,1639,"ahmed s, brennan l, eppig j, price cc, lamar m...",{'title': ['Comparative study between depressi...,visuoconstructional impairment in subtypes of ...,10.1111/j.1442-2018.2011.00666.x,['Comparative study between depression in Kore...,Comparative study between depression in Korean...,False,False,comparative study between depression in korean...,NaN,0,47.0,56.0,38.0,49.0,False,10.1016/j.jalz.2014.05.929,VISUOCONSTRUCTIONAL IMPAIRMENT IN SUBTYPES OF ...
55,10.1097/BOR.0b013e3283620163,563,"al-mossawi mh, ridley a, kiedel s, bowness p. ...",{'title': ['Gamma delta T cells in non-immune ...,"the role of natural killer cells, gamma delta ...",10.1002/iid3.18,['Gamma delta T cells in non-immune patients d...,Gamma delta T cells in non-immune patients dur...,False,False,gamma delta t cells in non-immune patients dur...,42.0,0,47.0,54.0,33.0,55.0,False,10.1097/bor.0b013e3283620163,"The role of natural killer cells, gamma delta ..."
72

In [180]:
df1_out = df1.merge(df_retry, how='left',)

In [181]:
df1_out.to_csv('./A4out_CrossRef_FuzzyMatched_with_retry_07Aug19.csv')

### Note:  The 'badlist' references, with additional information from Crossref, were then manually checked, and this information was then cleaned to return a 'curated DOI' for these references.

##### File names :  A5_badlist_checkedafter_API_retry(08Aug19).xls     and    A6in_currated_DOIs.csv.